<a href="https://colab.research.google.com/github/ErandySV/Simulaci-n-I/blob/main/Sistema_de_colas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Sistema de colas


Se tiene un sistema de colas formado por dos estaciones en serie, los clientes atendidos en la primera estación pasan enseguida a formar cola en la segunda en la primera estación de servicio la razón de llegadas sigue una distribución de poisson con media de $20$ clientes por hora y el tiempo de servicios sigue distribución exponencial con media de $2$ minutos por persona en la segunda estación el tiempo de servicio está uniformemente distribuido entre 1 y 2 minutos para esta información
¿Cuál es el tiempo promedio en el sistema y cuál de las dos colas que se forman es mayor?

Solucion analitica

Para resolver este problema de colas en serie, vamos a analizar cada estación de servicio por separado.

**Estación 1**

*   Tasa de llegadas   λ: 20 clientes por hora.
*   Tiempo de servicio: Distribución exponencial con media de 2 minutos por cliente.


Para convertir la media de servicio a horas:

Media del tiempo de servicio = $\frac{2}{60} = \frac{1}{30}$ horas por cliente

La tasa de servicio $(mu_1)$ es el inverso del tiempo de servicio medio:
$ \mu_1 = \frac{1}{\frac{1}{30}} = 30 \text{ clientes por hora} $

Tiempo promedio en la primera estación
Para un sistema \(M/M/1\), el tiempo promedio en el sistema \(W\) es:
$ W_1 = \frac{1}{\mu_1 - \lambda} = \frac{1}{30 - 20} = \frac{1}{10} \text{ horas}  \\
W_1 = \frac{1}{10} \times 60 = 6 \text{ minutos}$

**Estación 2**

*   Tasa de llegadas   λ = 20 clientes por hora
*   Tiempo de servicio:  Uniformemente distribuido entre 1 y 2 minutos.

El tiempo de servicio medio $mu_2$ para una distribución uniforme $[a, b]$ se calcula como:

Media del tiempo de servicio = $\frac{a + b}{2} = \frac{1 + 2}{2} = 1.5  minutos $

Convertimos a horas:
Media del tiempo de servicio= $\frac{1.5}{60} = \frac{1}{40}horas por cliente$

La tasa de servicio es:
$\mu_2 = \frac{1}{\frac{1}{40}} = 40  clientes por hora$

Tiempo promedio en la segunda estación

Para un sistema \(M/G/1\), el tiempo promedio en el sistema \(W\) es:
$ W_2 = \frac{1}{\mu_2 - \lambda} = \frac{1}{40 - 20} = \frac{1}{20} horas$
$ W_2 = \frac{1}{20} \times 60 = 3 \text{ minutos} $

**Tiempo promedio total en el sistema**

Sumamos los tiempos promedio en ambas estaciones:
$ W = W_1 + W_2 = 6 \text{ minutos} + 3 \text{ minutos} = 9 \text{ minutos}$

**Tamaño promedio de las colas **

Para calcular el tamaño promedio de las colas en cada estación, usamos la fórmula $L_q = \lambda W_q$, donde $W_q$ es el tiempo promedio de espera en la cola (sin incluir el servicio).

Para la estación 1 (usando \(M/M/1\)):

$ W_{q1} = \frac{\lambda}{\mu_1 (\mu_1 - \lambda)} = \frac{20}{30(30 - 20)} = \frac{20}{300} = \frac{1}{15} \text{ horas} $

$ W_{q1} = \frac{1}{15} \times 60 = 4 \text{ minutos} $

$ L_{q1} = \lambda W_{q1} = 20 \times \frac{1}{15} = \frac{20}{15} = 1.33  clientes$

Para la estación 2 (usando \(M/G/1\)):

$ W_{q2} = \frac{\lambda \sigma_s^2}{2(1 - \rho)} = \frac{20 \times \left(\frac{1}{12}\right)^2}{2(1 - 0.5)} = \frac{20 \times \frac{1}{144}}{2 \times 0.5} = \frac{20}{288} = \frac{5}{72}  horas $
$ W_{q2} = \frac{5}{72} \times 60 = \frac{25}{6} \text{ minutos} = 4.17 \text{ minutos} $
$ L_{q2} = \lambda W_{q2} = 20 \times \frac{5}{72} = \frac{100}{72} = 1.39 \text{ clientes}$

Por lo tanto:
    
       Tiempo promedio en el sistema: 9 minutos.
       Tamaño promedio de las colas:
                    Estación 1: 1.33 clientes.

                    Estación 2: 1.39 clientes.

La cola en la segunda estación es ligeramente mayor que en la primera estación.



In [ ]:
#Importamos librerias
import random as rnd
import pandas as pd

In [ ]:
# Función para generar tiempo exponencial
def generar_tiempo_exponencial(tasa):
    return rnd.expovariate(tasa)

In [ ]:
# Función para generar tiempo uniforme
def generar_tiempo_uniforme(minimo, maximo):
    return rnd.uniform(minimo, maximo)

 **Generar clientes:**
 * Se generan clientes a una tasa de llegada de 20 clientes por hora.
 * El tiempo entre llegadas de clientes se genera utilizando la distribución exponencial con media de 2 minutos.

 ** Atender a los clientes en la primera estación:**
 * Cuando un cliente llega a la primera estación, se verifica si el servidor está desocupado.
 * Si el servidor está desocupado, el cliente es atendido de inmediato.
 * El tiempo de servicio se genera utilizando la distribución exponencial con media de 2 minutos.
 * Si el servidor está ocupado, el cliente se coloca en la cola de la primera estación.
 **Atender a los clientes en la segunda estación:**
 * Cuando un cliente termina de ser atendido en la primera estación, pasa a la segunda estación.
 * Se verifica si el servidor de la segunda estación está desocupado.
 * Si el servidor de la segunda estación está desocupado, el cliente es atendido de inmediato.
 * El tiempo de servicio se genera utilizando la distribución uniforme entre 1 y 2 minutos.
 * Si el servidor de la segunda estación está ocupado, el cliente se coloca en la cola de la segunda estación.

In [ ]:
# Simulación de las colas
def simulacion_colas_dos_estaciones(num_clientes, tasa_llegadas, mu1, min_servicio_est2, max_servicio_est2):
    tiempo = 0
    clientes_en_cola_1 = 0
    clientes_en_cola_2 = 0
    tiempo_total = []
    llegada_clientes = []
    inicio_servicio_1 = []
    fin_servicio_1 = []
    inicio_servicio_2 = []
    fin_servicio_2 = []

    for i in range(num_clientes):
        tiempo_entre_llegadas = generar_tiempo_exponencial(tasa_llegadas)
        tiempo += tiempo_entre_llegadas
        llegada_clientes.append(tiempo)

        if clientes_en_cola_1 == 0:
            inicio_1 = tiempo
            tiempo_servicio_1 = generar_tiempo_exponencial(mu1)
            fin_1 = inicio_1 + tiempo_servicio_1
            tiempo = fin_1
        else:
            inicio_1 = max(tiempo, fin_servicio_1[-1])
            tiempo_servicio_1 = generar_tiempo_exponencial(mu1)
            fin_1 = inicio_1 + tiempo_servicio_1
            tiempo = fin_1
            clientes_en_cola_1 -= 1

        inicio_servicio_1.append(inicio_1)
        fin_servicio_1.append(fin_1)

        if clientes_en_cola_2 == 0:
            inicio_2 = fin_1
            tiempo_servicio_2 = generar_tiempo_uniforme(min_servicio_est2, max_servicio_est2)
            fin_2 = inicio_2 + tiempo_servicio_2
            tiempo = fin_2
        else:
            inicio_2 = max(fin_1, fin_servicio_2[-1])
            tiempo_servicio_2 = generar_tiempo_uniforme(min_servicio_est2, max_servicio_est2)
            fin_2 = inicio_2 + tiempo_servicio_2
            tiempo = fin_2
            clientes_en_cola_2 -= 1

        inicio_servicio_2.append(inicio_2)
        fin_servicio_2.append(fin_2)
        tiempo_total.append(fin_2 - llegada_clientes[-1])

    data = {
        'Cliente': list(range(1, num_clientes + 1)),
        'Llegada (min)': [t * 60 for t in llegada_clientes],
        'Inicio Servicio Est1 (min)': [t * 60 for t in inicio_servicio_1],
        'Fin Servicio Est1 (min)': [t * 60 for t in fin_servicio_1],
        'Inicio Servicio Est2 (min)': [t * 60 for t in inicio_servicio_2],
        'Fin Servicio Est2 (min)': [t * 60 for t in fin_servicio_2],
        'Tiempo en el Sistema (min)': [t * 60 for t in tiempo_total]
    }
    df = pd.DataFrame(data)
    print(df)

    tiempo_promedio_sistema = sum(tiempo_total) / num_clientes
    longitud_promedio_cola_1 = sum(max(0, inicio_servicio_1[i + 1] - fin_servicio_1[i]) for i in range(num_clientes - 1)) / num_clientes
    longitud_promedio_cola_2 = sum(max(0, inicio_servicio_2[i + 1] - fin_servicio_2[i]) for i in range(num_clientes - 1)) / num_clientes

    return {
        'tiempo_promedio_sistema': tiempo_promedio_sistema,
        'longitud_promedio_cola_1': longitud_promedio_cola_1,
        'longitud_promedio_cola_2': longitud_promedio_cola_2,
        'tabla_resultados': df
    }


In [ ]:
# Cálculo analitico para la comparación con simulación
def calculo_analitico(llegadas_est1, media_servicio_est1, media_servicio_est2):
    # Tasa de servicio
    tasa_servicio_est1 = 1 / media_servicio_est1
    tasa_servicio_est2 = 1 / media_servicio_est2

    # Utilización
    rho1 = llegadas_est1 / tasa_servicio_est1
    rho2 = llegadas_est1 / tasa_servicio_est2  # Igual a la tasa de llegada de la segunda estación

    # Tiempo promedio en el sistema
    W1 = 1 / (tasa_servicio_est1 - llegadas_est1)
    W2 = 1 / (tasa_servicio_est2 - llegadas_est1)

    # Convertir a minutos
    W1_min = W1 * 60
    W2_min = W2 * 60
    W_total = W1_min + W2_min

    # Tiempo promedio de espera en la cola (sin incluir el servicio)
    Wq1 = W1 - media_servicio_est1
    Wq2 = (llegadas_est1 ** 2 * ((1 / 60) ** 2 / 12) + (llegadas_est1 / tasa_servicio_est2) ** 2) / (2 * (1 - rho2))

    # Tamaño promedio de la cola
    Lq1 = llegadas_est1 * Wq1
    Lq2 = llegadas_est1 * Wq2

    return {
        'W1_min': W1_min,
        'W2_min': W2_min,
        'W_total': W_total,
        'Lq1': Lq1,
        'Lq2': Lq2
    }


 Definir los parámetros:
 * Tasa de llegada de clientes (λ): 20 clientes por hora
 * Tiempo medio de servicio en la primera estación (μ1): 2 minutos por persona
 * Tiempo medio de servicio en la segunda estación (μ2): 1.5 minutos por persona
 * Número de servidores en la primera estación (c1): 1
 * Número de servidores en la segunda estación (c2): 1

In [39]:

num_clientes = 1000
llegadas_est1 = 20  # clientes por hora
media_servicio_est1 = 2 / 60  # horas
media_servicio_est2 = 1.5 / 60  # horas
min_servicio_est2 = 1 / 60  # horas
max_servicio_est2 = 2 / 60  # horas

In [41]:


# Ejecutar la simulación
resultados_simulacion = simulacion_colas_dos_estaciones(num_clientes, llegadas_est1 / 60, 1 / media_servicio_est1, min_servicio_est2, max_servicio_est2)

# Calcular resultados analíticos
resultados_analiticos = calculo_analitico(llegadas_est1, media_servicio_est1, media_servicio_est2)

     Cliente  Llegada (min)  Inicio Servicio Est1 (min)  \
0          1     597.914863                  597.914863   
1          2     676.875794                  676.875794   
2          3     915.816958                  915.816958   
3          4    1483.359127                 1483.359127   
4          5    1697.638793                 1697.638793   
..       ...            ...                         ...   
995      996  174367.658158               174367.658158   
996      997  174545.969040               174545.969040   
997      998  174752.972804               174752.972804   
998      999  174814.731662               174814.731662   
999     1000  174857.766616               174857.766616   

     Fin Servicio Est1 (min)  Inicio Servicio Est2 (min)  \
0                 598.283616                  598.283616   
1                 680.252773                  680.252773   
2                 917.468238                  917.468238   
3                1484.595473                 1484.5

 Analisi de  los resultados:
 * Se calcula el tiempo promedio que un cliente pasa en el sistema.
 * Se calcula la longitud promedio de la cola en cada estación.

In [42]:
# Mostrar resultados de la simulación
print("Resultados de la Simulación:")
print(f"Tiempo promedio en el sistema: {resultados_simulacion['tiempo_promedio_sistema'] * 60:.2f} minutos")
print(f"Longitud promedio de la cola en la primera estación: {resultados_simulacion['longitud_promedio_cola_1']:.2f} clientes")
print(f"Longitud promedio de la cola en la segunda estación: {resultados_simulacion['longitud_promedio_cola_2']:.2f} clientes")

# Mostrar resultados analíticos
print("\nResultados Analíticos:")
print(f"Tiempo promedio en el sistema: {resultados_analiticos['W_total']:.2f} minutos")
print(f"Tamaño promedio de la cola en la primera estación: {resultados_analiticos['Lq1']:.2f} clientes")
print(f"Tamaño promedio de la cola en la segunda estación: {resultados_analiticos['Lq2']:.2f} clientes")

# Comparación
if resultados_analiticos['Lq1'] > resultados_analiticos['Lq2']:
    print("\nLa cola en la primera estación es mayor que en la segunda estación.")
else:
    print("\nLa cola en la segunda estación es mayor que en la primera estación.")


Resultados de la Simulación:
Tiempo promedio en el sistema: 3.48 minutos
Longitud promedio de la cola en la primera estación: 2.87 clientes
Longitud promedio de la cola en la segunda estación: 2.88 clientes

Resultados Analíticos:
Tiempo promedio en el sistema: 9.00 minutos
Tamaño promedio de la cola en la primera estación: 1.33 clientes
Tamaño promedio de la cola en la segunda estación: 5.19 clientes

La cola en la segunda estación es mayor que en la primera estación.
